## 实验：Huggingface Transformer & Diffusion

#### 准备工作

1. 创建虚拟环境：

```
conda create -n huggingface
conda activate huggingface
conda install python=3.11 notebook ipywidgets
```

2. 设置 Pip 软件源，加快下载速度：

```
pip config set global.index-url https://mirrors.bfsu.edu.cn/pypi/web/simple
```

2. 根据硬件选择合适的 PyTorch 版本（二选一）：

- 选项1：CPU 版 PyTorch

```
pip3 install torch torchvision
```

- 选项2：GPU 版 PyTorch

```
pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu118
```

3. 安装 Huggingface 相关软件库：

```
pip install diffusers["torch"] transformers huggingface_hub
```

#### 开始实验！

情感分析：

In [ ]:
from transformers import pipeline
pipe = pipeline(task = "text-classification")

In [ ]:
pipe("Professor Qiu is very strict.")

In [ ]:
pipe("There is a new coffee shop on campus.")

In [ ]:
pipe("There is a new coffee shop on campus. The drinks there taste not bad.")

In [ ]:
pipe("There is a new coffee shop on campus. I will never go there again.")

填词：

In [ ]:
fill_masker = pipeline(task = "fill-mask", model = "ethanyt/guwenbert-base")

In [ ]:
output = fill_masker("帘外雨潺潺，春意阑珊。罗[MASK]不耐五更寒。")
list(map(lambda x : (x["token_str"], x["score"]), output))

文生图（CPU 版，可能会非常慢）：

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float32
)

第一次运行时代码会下载模型文件，如果出现连接问题，可以用如下命令手动下载（该命令会下载所有版本的模型文件，非常大，如果不需要完整版可以去往 https://aliendao.cn/models/runwayml/stable-diffusion-v1-5 手动下载）：

```
python model_download.py --repo_id runwayml/stable-diffusion-v1-5 --mirror
```

然后加载本地模型文件：

```python
import torch
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "./dataroot/models/runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float32
)
```

In [ ]:
prompt = "A blue apple"
pipe.enable_attention_slicing()
image = pipe(prompt, num_inference_steps=10).images[0]

In [ ]:
image

GPU 版：

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe.to("cuda")

In [ ]:
prompt = "A tree on the moon"
pipe.enable_attention_slicing()
image = pipe(prompt, num_inference_steps=50).images[0]

In [ ]:
image

控制随机数种子：

In [ ]:
gen = torch.Generator(device="cuda").manual_seed(123)
image = pipe(prompt, generator=gen, num_inference_steps=50).images[0]
image